In [1]:
import deeplabcut

import yaml

from paramiko import SSHClient
import paramiko
from scp import SCPClient
import getpass

import os
import glob

In [2]:
# set shrek project folder and local folder (dest_folder)

shrek_project_folder = '/home/jts3256/projects/Han_202102-Joe-2021-02-04'
dest_folder = r'D:\Lab\Data\DLC_videos\Han_20210211_rwFreeReach'

In [ ]:
# move folder from Shrek to local computer to label frames on local computer. Set shrek project folder and dest_folder
# Also may need to update ssh key/id
image_folder = shrek_project_folder + '/labeled-data'
config_file = shrek_project_folder + '/config.yaml'

if(not os.path.isdir(dest_folder)):
    os.mkdir(dest_folder)

ssh = SSHClient()
ssh.load_host_keys(filename = r'C:\Users\Joseph Sombeck\shrek_publickey (2).ppk')

pkey_filename = r'C:\Users\Joseph Sombeck\shrek_sshkey'
#pkey = paramiko.RSAKey.from_private_key_file(pkey_filename)
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())



ssh.connect(hostname = "FSMS270859X8810459.fsm.northwestern.edu", username= "jts3256", key_filename=pkey_filename)
with SCPClient(ssh.get_transport(), sanitize=lambda x: x) as scp:
    scp.get(image_folder,recursive = True, local_path=dest_folder)
    scp.get(config_file,recursive = False, local_path=dest_folder)
    
local_config_path = dest_folder+ r'\config.yaml'

# update project path in config path
with open(local_config_path) as file:
    data = yaml.load(file, Loader = yaml.FullLoader)

    data['project_path'] = dest_folder # default = 20

with open(local_config_path,'w') as file:
    yaml.dump(data,file)


In [3]:
local_config_path

'D:\\Lab\\Data\\DLC_videos\\Han_20210621_freeReach\\config.yaml'

In [3]:
"""
label frames
"""
local_config_path= r'D:\Lab\Data\DLC_videos\Han_20210621_freeReach\config.yaml'
    
#local_config_path = dest_folder + r'\config.yaml'

deeplabcut.label_frames(local_config_path,imtypes=['*png'])

RuntimeError: wrapped C/C++ object of type DirDialog has been deleted

In [ ]:
#deeplabcut.check_labels(local_config_path)

# move files back onto shrek
ssh = SSHClient()
ssh.load_host_keys(filename = r'C:\Users\Joseph Sombeck\shrek_publickey (2).ppk')

pkey_filename = r'C:\Users\Joseph Sombeck\shrek_sshkey'
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect(hostname = "FSMS270859X8810459.fsm.northwestern.edu", username= "jts3256", key_filename=pkey_filename)
scp  = SCPClient(ssh.get_transport())

scp.put(files = dest_folder + '\labeled-data',recursive = True, 
        remote_path =shrek_project_folder)
